In [1]:
import numpy as np
import trimesh
import scipy

In [10]:
# create test and train set 
# save it as txt files
import glob
import random

# testset = []
# with open('data/testset.txt') as f:
#     testset = f.read().splitlines()
files = glob.glob('data/new_data/laplacian/*.laplacian')
old = []
new = []
for file in files:
    file = file.split('/')[-1].split('.')[0]
    if file.startswith("meshes"):
        old.append(file)
    else:
        new.append(file)

# print(len(new), len(old)) ###48 169
random.shuffle(old)
random.shuffle(new)
trainset = [] #28 + 122
valset = [] #12 + 35
testset = [] # 8 + 12

for i, path in enumerate(old):
    if i<122:
        trainset.append(path)
    elif i < 157:
        valset.append(path)
    else:
        testset.append(path)
# for i, path in enumerate(new):
#     if i<28:
#         trainset.append(path)
#     elif i < 12:
#         valset.append(path)
#     else:
#         testset.append(path)

print(len(trainset))
# testset = set(testset)
# def get_random_train_list():
#     files = glob.glob('data/xyz/*.xyz')
#     train = random.sample(files, 120)
#     train_set = set(train)
#     train_list = []
#     val_list = []
#     for file in files:
#         name = file.split('/')[-1].split('.')[0]
#         if file in train_set:
#             train_list.append(name)
#         elif name not in testset:
#             val_list.append(name)

#     return train_list, val_list
def write_list_to_file(filename,my_list):
    with open(filename, 'w') as f:
        for item in my_list:
            f.write("%s\n" % item)
# train_list, val_list = get_random_train_list()
write_list_to_file('data/new_data/trainingset.txt', trainset)
write_list_to_file('data/new_data/valset.txt', valset)
write_list_to_file('data/new_data/testset.txt', testset)

122


In [95]:
from random import sample
import glob
import trimesh
def save_xyz(pts, file_name):
    # print(pts)
    s = trimesh.util.array_to_string(pts)
    with open(file_name, 'w') as f:
        f.write("%s\n" % s)
def sample_k(matrix,k):
    # matrix = matrix.toarray()
    result = []
    for i,row in enumerate(matrix):
        row = list(row)
        if len(row) >= k:
            k_nearest = sample(row,k)
        else:
            while len(row) < k:
                row.append(0.0)
            k_nearest = row
        result.append(k_nearest)
    return result
def save_vertices():
    files = glob.glob('data/noisy/*.obj')
    for file in files:
        print(file)
        mesh = trimesh.load_mesh(file)
        vertices = mesh.vertices
        dest_name = 'data/xyz/' +file.split('/')[-1].split('.')[0] +'.xyz'
        save_xyz(vertices, dest_name)
        
def create_target():
    files = glob.glob('data/noisy/*.obj')
    for file in files:
        print(file)
        mesh = trimesh.load_mesh(file)
        result = knn_cotangent_laplacian(mesh, 6)
    
        dest_name = 'data/laplacian/' +file.split('/')[-1].split('.')[0] +'.laplacian'
        save_xyz(result, dest_name)
        break

create_target()


data/noisy/meshes_march2_mesh_0120.obj


In [2]:
from scipy.sparse import coo_matrix, eye
from scipy.spatial import distance
def get_cotangent_weight(i,neighbor,neighbors,norms, KNN):
    def get_k_h(i,j):
        list1, list2 = neighbors[i], neighbors[j]
        #if there is morethan 2 intersection choose the first 2 points
        intersection = list(set(list1).intersection(list2))
        try:
            k,h = intersection
        except:
            k,h = intersection[:2]
            # print(intersection)
            # print(i,j)
            # print('k,h', )

        return k,h
    def get_distance(i,j,k,h):
        j_idx = neighbors[i].index(j)
        lij = norms[i][j_idx]

        k_idx = neighbors[j].index(k)
        ljk = norms[j][k_idx]

        i_idx = neighbors[k].index(i)
        lki = norms[k][i_idx]

        h_idx = neighbors[j].index(h)
        ljh = norms[j][h_idx]

        i_idx = neighbors[h].index(i)
        lhi = norms[h][i_idx]

        return lij, ljk, lki, ljh, lhi 

    result = []
    for j in KNN:
        #if j in neighbor find Cotangent weight else assign 0
        wij = 0
        if j in neighbor:
            
            k,h  = get_k_h(i,j)
            lij, ljk, lki, ljh, lhi = get_distance(i,j,k,h)

            s_ijk = (lij + ljk + lki)/2
            A_ijk = 8 *  np.sqrt(s_ijk * ( s_ijk - lij) * ( s_ijk- ljk) * ( s_ijk - lki))

            s_ijh = (lij + ljh + lhi)/2
            A_ijh = 8 *  np.sqrt(s_ijh * ( s_ijh - lij) * ( s_ijh- ljh) * ( s_ijh - lhi))

            
            wij = ((-lij**2 + ljk ** 2 + lki**2)/  A_ijk)  + ((-lij**2 + ljh ** 2 + lhi**2)/  A_ijh)
        
       
        result.append(wij)
        
    return result

def knn_cotangent_laplacian(mesh, k):
    neighbors = mesh.vertex_neighbors
    
    vertices = mesh.vertices.view(np.ndarray)

    ones = np.ones(3)
    norms = [np.sqrt(np.dot((vertices[i] - vertices[n]) ** 2, ones))
                for i, n in enumerate(neighbors)]
    D = distance.squareform(distance.pdist(vertices))
    closest = np.argsort(D, axis=1)
    closest = closest[:, 1:k+1]



    # norms = [i / i.sum() for i in norms]
    data = []

    for i, KNN in enumerate(closest):
        neighbor = neighbors[i]
        weight = get_cotangent_weight(i,neighbor, neighbors,norms, KNN)
        data.append(weight)
        # create the sparse matrix
    
    col = np.concatenate(closest)
    row = np.concatenate([[i] * len(n)
                          for i, n in enumerate(closest)])
    
   

    data = np.concatenate([i / np.array(i).sum() if np.array(i).sum()>0 else i for i in data])
    
    
    matrix = coo_matrix((data, (row, col)),
                        shape=[len(vertices)] * 2)
    return matrix

In [3]:
from scipy.sparse import coo_matrix, eye
from scipy.spatial import distance
def get_pred_matrix(mesh, k, fname):
    neighbors = mesh.vertex_neighbors
    
    vertices = mesh.vertices.view(np.ndarray)

    ones = np.ones(3)
    norms = [np.sqrt(np.dot((vertices[i] - vertices[n]) ** 2, ones))
                for i, n in enumerate(neighbors)]
    D = distance.squareform(distance.pdist(vertices))
    closest = np.argsort(D, axis=1)
    closest = closest[:, 1:k+1]



    # norms = [i / i.sum() for i in norms]
    data = np.genfromtxt(fname, delimiter=' ')
    

    # for i, KNN in enumerate(closest):
    #     neighbor = neighbors[i]
    #     weight = text_array()
    #     data.append(weight)
    #     # create the sparse matrix
    
    col = np.concatenate(closest)
    row = np.concatenate([[i] * len(n)
                          for i, n in enumerate(closest)])
    
   

    data = np.concatenate([i / np.array(i).sum() if np.array(i).sum()>0 else i for i in data])
    
    
    matrix = coo_matrix((data, (row, col)),
                        shape=[len(vertices)] * 2)
    return matrix
def make_smooth(mesh, matrix= None):
    if matrix != None:
        matrix = matrix
    else:
        # print("Matrix = None")
        matrix = trimesh.smoothing.laplacian_calculation(mesh, equal_weight= True)
        
    smoothed_mesh = trimesh.smoothing.filter_taubin(mesh, lamb=0.4, nu=0.5, iterations=10, laplacian_operator=matrix)
                    
    return smoothed_mesh
def smooth_bunch():
    with open('data/eval.txt') as f:
        for line in f:
            line = line.strip('\n') + '.obj'
            mesh_path = 'data/noisy/'+line
            mesh = trimesh.load_mesh(mesh_path)
            fname = 'data/laplacian/' + line.split('.')[0] +'.laplacian'
            # matrix = get_pred_matrix(mesh, 6, fname)
            smoothed =  make_smooth(mesh)
            outpath = 'output/filter_laplacian/'+line
            output = smoothed.export(outpath)
            print(line)
            # break


In [8]:
mesh_path = 'data/noisy/meshes_march1_mesh_0080.obj'
lapl_path = 'data/laplacian/meshes_march1_mesh_0080.laplacian'
mesh = trimesh.load_mesh(mesh_path)
# laplacian = knn_cotangent_laplacian(mesh, 6)
laplacian = get_pred_matrix(mesh, k=6, fname=lapl_path)
mesh = make_smooth(mesh,matrix= laplacian)
mesh.show()


In [52]:
mesh1, mesh2 = 'output/meshes_bouncing_mesh_0020_smooth_ctg.obj', 'output/meshes_bouncing_mesh_0020_smooth.obj'
calculate_loss(mesh2, smooth_path)
#(tensor(0.0778), tensor(0.1726))
#(tensor(0.0915), tensor(0.1714))
# (tensor(0.0392), tensor(0.1713))

#(tensor(0.0394), tensor(0.1700)) ctg, smooth
# (tensor(0.0948), tensor(0.1706)) normal, smooth

loss(smooth_gt,noisy_gt)
ChamferDistance:  tensor(0.0948)
MSE:  tensor(0.1706)


(tensor(0.0948), tensor(0.1706))

In [9]:
import torch
import numpy as np
from sklearn.neighbors import NearestNeighbors
from chamferdist import ChamferDistance
import glob
def calculate_loss(mesh1, mesh2):
    loss = ChamferDistance()
    mse_loss = torch.nn.MSELoss()
    mesh1 = trimesh.load_mesh(mesh1)
    mesh2 = trimesh.load_mesh(mesh2)
    v1, v2 = mesh1.vertices.view(np.ndarray),mesh2.vertices.view(np.ndarray)
    v1, v2  = torch.tensor(v1).float().unsqueeze(0), torch.tensor(v2).float().unsqueeze(0)
    
    chamfer_loss = loss(v1, v2)
    mse_loss = 0
    # print('===================================')
    # print('loss(smooth_gt,noisy_gt)')
    # print('ChamferDistance: ', chamfer_loss)
    # print('MSE: ', mse_loss)
    # print('===================================')
    return chamfer_loss, mse_loss

# mesh1 = 'data/smooth/meshes_bouncing_mesh_0020.obj'
# mesh2 = 'data/noisy/meshes_bouncing_mesh_0020.obj'

# chamfer, mse = calculate_loss(outpath, mesh1)
def loss_bunch():
    files = glob.glob('data/remote/pred/*.obj')
    
    chamfer_total, mse_total = 0, 0
    for file in files:
        mesh1 = 'data/remote/smooth/'+file.split('/')[-1]
        mesh2 = 'data/remote/pred/'+file.split('/')[-1]
        chamfer, mse = calculate_loss(mesh1, mesh2)
        chamfer_total += chamfer
        mse_total += mse
        print(file)
        print('<br>Chamfer: ', float(chamfer).__round__(4),'  MSE: ', float(mse).__round__(4), '<br>')
    # chamfer_total, mse_total = chamfer_total/len(files), mse_total/len(files)
    # print('ChamferDistance: ', chamfer_total)
    # print('MSE: ', mse_total)

loss_bunch()

data/remote/pred/chinese-lion_n2.obj
<br>Chamfer:  1360.1971   MSE:  0.0 <br>
data/remote/pred/girl-scanned_n1.obj
<br>Chamfer:  84378.0625   MSE:  0.0 <br>


In [20]:
#eigenvalue
from scipy.sparse.linalg import eigs
mesh_path = 'data/noisy/meshes_march2_mesh_0120.obj'
mesh = trimesh.load_mesh(mesh_path)
matrix_cotangent = trimesh.smoothing.laplacian_calculation(smooth_mesh,6)
# id = np.eye(13)
vals, vecs = eigs(matrix_cotangent, k=6)
vecs.shape

(10002, 6)

In [4]:
smooth_mesh = trimesh.load_mesh('data/smooth/meshes_march2_mesh_0120.obj')
smooth_laplacian = trimesh.smoothing.laplacian_calculation(smooth_mesh)


In [6]:
noisy_mesh = trimesh.load_mesh('data/noisy/meshes_march2_mesh_0120.obj')
smooth_mesh = make_smooth(noisy_mesh,matrix= smooth_laplacian)
smooth_mesh.show()